# A New Enron Feature Quiz

In [1]:
import os
import zipfile
import reader
import poi_emails

/Users/snewman/.Envs/sandbox/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def getToFromStrings(f):
    '''
    The imported reader.py file contains functions that we've created to help
    parse e-mails from the corpus. .getAddresses() reads in the opening lines
    of an e-mail to find the To: From: and CC: strings, while the
    .parseAddresses() line takes each string and extracts the e-mail addresses
    as a list.
    '''
    f.seek(0)
    to_string, from_string, cc_string   = reader.getAddresses(f)
    to_emails   = reader.parseAddresses( to_string )
    from_emails = reader.parseAddresses( from_string )
    cc_emails   = reader.parseAddresses( cc_string )

    return to_emails, from_emails, cc_emails


def poiFlagEmail(f):
    """ given an email file f,
        return a trio of booleans for whether that email is
        to, from, or cc'ing a poi """

    to_emails, from_emails, cc_emails = getToFromStrings(f)

    ### poi_emails.poiEmails() returns a list of all POIs' email addresses.
    poi_email_list = poi_emails.poiEmails()

    to_poi = False
    from_poi = False
    cc_poi   = False

    ### to_poi and cc_poi are boolean variables which flag whether the email
    ### under inspection is addressed to a POI, or if a POI is in cc,
    ### respectively. You don't have to change this code at all.

    ### There can be many "to" emails, but only one "from", so the
    ### "to" processing needs to be a little more complicated
    if to_emails:
        ctr = 0
        while not to_poi and ctr < len(to_emails):
            if to_emails[ctr] in poi_email_list:
                to_poi = True
            ctr += 1

    if cc_emails:
        ctr = 0
        while not cc_poi and ctr < len(cc_emails):
            if cc_emails[ctr] in poi_email_list:
                cc_poi = True
            ctr += 1

    #################################
    ######## your code below ########
    ### set from_poi to True if #####
    ### the email is from a POI #####
    #################################
    if from_emails:
        ctr = 0
        while not from_poi and ctr < len(from_emails):
            if from_emails[ctr] in poi_email_list:
                from_poi = True
            ctr += 1

    #################################
    return to_poi, from_poi, cc_poi

# Visualizing Your New Feature

In [3]:
import pickle
from get_data import getData

def computeFraction( poi_messages, all_messages ):
    """ given a number messages to/from POI (numerator) 
        and number of all messages to/from a person (denominator),
        return the fraction of messages to/from that person
        that are from/to a POI
    """
    if all_messages == 'NaN' or poi_messages == 'NaN':
        fraction = 0.0
    else:
        fraction = float(poi_messages) / float(all_messages)
    
    return fraction

data_dict = getData() 
submit_dict = {}

for name in data_dict:

    data_point = data_dict[name]

    print
    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
    print fraction_from_poi
    data_point["fraction_from_poi"] = fraction_from_poi


    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
    print fraction_to_poi
    submit_dict[name]={"from_poi_to_this_person":fraction_from_poi,
                       "from_this_person_to_poi":fraction_to_poi}

    data_point["fraction_to_poi"] = fraction_to_poi
    
    
#####################

def submitDict():
    return submit_dict



0.0470879801735
0.0344827586207

0.0
0.0

0.0
0.0

0.0130890052356
0.0

0.0306220095694
0.65625

0.0
0.0

0.0
0.0

0.0246284501062
0.541666666667

0.0187234042553
0.0139794967381

0.0492730210016
0.216216216216

0.078125
1.0

0.108108108108
0.0

0.010101010101
0.142857142857

0.013978088402
0.342105263158

0.0
0.0

0.0
0.0

0.136518771331
0.275

0.0882352941176
0.0

0.0968992248062
0.339285714286

0.0
0.0

0.0
0.0

0.0253353204173
0.0

0.0302227573751
0.112268518519

0.0
0.0

0.0
0.0

0.0291834833903
0.0309585975382

0.0
0.0

0.0104438642298
0.0

0.0
0.0

0.014312383323
0.2

0.0263554216867
0.585365853659

0.0
0.0

0.0196855775803
0.0888786553074

0.00698080279232
0.368421052632

0.0
0.0

0.0
0.0

0.030303030303
0.037037037037

0.0
0.0

0.0105042016807
0.0

0.031746031746
0.0

0.0174459176553
0.0093023255814

0.0
0.0

0.0237420269313
0.222222222222

0.072737291638
0.158994197292

0.0
0.0

0.0
0.0

0.0
0.0

0.0
0.0

0.0
0.0

0.0689045936396
0.0

0.0
0.0

0.00488481087861
0.053497942386

# Number of Features and Overfitting

In [4]:
import pickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

This is `find_signature.py`:

In [9]:
numpy.random.seed(42)

words_file = "your_word_data.pkl" 
authors_file = "your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )

features_train, features_test, labels_train, labels_test = \
   cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test).toarray()

features_train = features_train[:150].toarray()
labels_train = labels_train[:150]

clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

In [10]:
print len(features_train)

150


# Accuracy of Your Overfit Decision Tree

In [11]:
print clf.score(features_test, labels_test)

0.947667804323


# Identify the Most Powerful Features

In [16]:
importances = [i for i in clf.feature_importances_ if i >= 0.2]

print importances
print list(clf.feature_importances_).index(importances[0])

[0.76470588235294124]
33614


# Use TfIdf to Get the Most Important Word

In [19]:
print vectorizer.get_feature_names()[33614]

sshacklensf


# Remove, Repeat

In [28]:
import os
import pickle
import re
import string

from nltk.stem.snowball import SnowballStemmer

from_sara  = open('from_sara.txt', 'r')
from_chris = open('from_chris.txt', 'r')

from_data = []
word_data = []

def parseOutText(f):
    stemmer = SnowballStemmer('english')
    f.seek(0) # go back to beginning of file
    all_text = f.read()
    content = all_text.split('X-FileName:') # split off metadata
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(string.maketrans("", ""), string.punctuation)
        words = text_string.split()
        stemmed_words = [stemmer.stem(w) for w in words]
    return ' '.join(stemmed_words)
    
### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0

for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        if temp_counter < 200:
            path = os.path.join('../../enron_email', path[:-1])
            email = open(path, "r")
            words = parseOutText(email)
            words = words.replace('sara', '')
            words = words.replace('shackleton', '')
            words = words.replace('chris', '')
            words = words.replace('germani', '')
            words = words.replace('sshacklensf', '')
            word_data.append(words)
            if name == "sara":
                from_data.append("0")
            else:
                from_data.append("1")
            email.close()

print "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_updated_word_data.pkl", "w") )
pickle.dump( from_data, open("your_updated_email_authors.pkl", "w") )

emails processed


Rerun `find_signature.py`:

In [29]:
numpy.random.seed(42)

words_file = "your_updated_word_data.pkl" 
authors_file = "your_updated_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )

features_train, features_test, labels_train, labels_test = \
   cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test).toarray()

features_train = features_train[:150].toarray()
labels_train = labels_train[:150]

clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

importances = [i for i in clf.feature_importances_ if i > 0.2]

print importances
idx = list(clf.feature_importances_).index(importances[0])
print vectorizer.get_feature_names()[idx]

[0.66666666666666674]
cgermannsf


# Checking Important Features Again

In [31]:
import os
import pickle
import re
import string

from nltk.stem.snowball import SnowballStemmer

from_sara  = open('from_sara.txt', 'r')
from_chris = open('from_chris.txt', 'r')

from_data = []
word_data = []

def parseOutText(f):
    stemmer = SnowballStemmer('english')
    f.seek(0) # go back to beginning of file
    all_text = f.read()
    content = all_text.split('X-FileName:') # split off metadata
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(string.maketrans("", ""), string.punctuation)
        words = text_string.split()
        stemmed_words = [stemmer.stem(w) for w in words]
    return ' '.join(stemmed_words)
    
### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0

for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        if temp_counter < 200:
            path = os.path.join('../../enron_email', path[:-1])
            email = open(path, "r")
            words = parseOutText(email)
            words = words.replace('sara', '')
            words = words.replace('shackleton', '')
            words = words.replace('chris', '')
            words = words.replace('germani', '')
            words = words.replace('sshacklensf', '')
            words = words.replace('cgermannsf', '')
            word_data.append(words)
            if name == "sara":
                from_data.append("0")
            else:
                from_data.append("1")
            email.close()

print "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_updated2_word_data.pkl", "w") )
pickle.dump( from_data, open("your_updated2_email_authors.pkl", "w") )

emails processed


In [33]:
numpy.random.seed(42)

words_file = "your_updated2_word_data.pkl" 
authors_file = "your_updated2_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )

features_train, features_test, labels_train, labels_test = \
   cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test).toarray()

features_train = features_train[:150].toarray()
labels_train = labels_train[:150]

clf = DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

importances = [i for i in clf.feature_importances_ if i > 0.2]

print importances
print clf.feature_importances_
idx = list(clf.feature_importances_).index(importances[0])
print vectorizer.get_feature_names()[idx]

[0.36363636363636365]
[ 0.  0.  0. ...,  0.  0.  0.]
houectect


In [34]:
print clf.score(features_test, labels_test)

0.816837315131
